In [1]:
import sys
sys.path.append("../../")
sys.path.append("..")
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import HTML

In [2]:
from cellmating.tracking._hausdorf_tracker import Tracker
from cellmating.visualize import animate_images
from cellmating.utils import file_traverse
from cellmating.io import imread, imsave
from cellmating.utils import crop_single_object
from cellmating.visualize import label2rgb

%load_ext autoreload
%autoreload 2

In [6]:
file_root = "/media/wd/T7/data/mating_all/20220903_YAV2104xYAV2105_003/tif/"
# file_root = "/Volumes/T7/data/mating_all/20220903_YAV2104xYAV2105_003/tif/"
file_list = file_traverse(file_root, file_regular=".*._rigid.tif$")

In [114]:
filename = file_list[10]
print(filename)
image = imread(filename)
image = np.moveaxis(image, -1, 1)
print(image.shape)

/media/wd/T7/data/mating_all/20220903_YAV2104xYAV2105_003/tif/YAV2104xYAV2105_005_11_rigid.tif
(241, 4, 1020, 1020)


In [121]:
trace = Tracker(image[:, -1], max_miss=10, min_hist=3, low_ratio=0.5, high_ratio=1, is_acc=False)
_ = trace()
traced_image = trace.to_image()

frame= 1 [1010, 1243] [1244]
frame= 2 [] [1242]
frame= 3 [] [1243]
frame= 4 [1240] [1242]
frame= 5 [1240, 3002] [1224, 1241]
divison: [[0 0 1]]
fusion: [[1 2 0]]
frame= 6 [1125, 3002] [1225, 1241]
divison: [[0 0 1]]
frame= 7 [1240] [1225, 1244]
frame= 8 [1240, 1239] [1244, 1245]
frame= 9 [1010, 1126, 1240] [1243, 1244]
divison: [[0 0 1]]
frame= 10 [1126, 1240] [1086, 1246, 1247]
divison: [[0 0 1]]
frame= 11 [1240, 1242] [1085, 1226, 1245, 1246]
divison: [[0 0 1]]
frame= 12 [1240] [1087, 1247]
frame= 13 [1010, 1240] [1086, 1245]
divison: [[2 2 3]]
frame= 14 [1010, 1043, 1240, 1246] [1044, 1086, 1246]
divison: [[0 0 2]
 [1 1 3]
 [2 4 5]
 [3 7 8]]
frame= 15 [] [1093, 1255]
frame= 16 [1251] []
frame= 17 [1011] [1090, 1252]
divison: [[0 0 1]
 [1 4 5]]
fusion: [[0 1 2]]
frame= 18 [] [1049, 1091, 1252]
divison: [[0 0 1]
 [1 3 4]]
frame= 19 [1247] [1093, 1255]
frame= 20 [] [1049, 1092, 1256]
divison: [[0 0 2]]
frame= 21 [1134] [1092, 1255]
divison: [[1 1 2]]
frame= 22 [1134, 1252] [1092, 1255]

In [122]:
imsave(filename[:-4]+"_trackv2.tif", traced_image, dtype=np.uint16, imagej=True)

In [117]:
# annotation = label2rgb(traced_image)

In [118]:
# ani = animate_images(image[200:, -1,:,:])
# ani2 = animate_images(annotation[200:])

# html1 = HTML(ani.to_jshtml()).data
# html2 = HTML(ani2.to_jshtml()).data

# # Display the combined animations
# HTML(f"""
# <div style="display: flex;">
#     <div style="flex: 1; margin-right: 10px;">
#         {html1}
#     </div>
#     <div style="flex: 1; margin-left: 10px;">
#         {html2}
#     </div>
# </div>
# """)

In [108]:
from cellmating.utils import crop_object_to_dict
import json
from cellmating.utils import dump_to_pkl
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [109]:
labels = np.unique(traced_image[-1])[1:]
labels = list(labels[labels>2000])

In [100]:
labels = [i for i in labels if i not in [3431, 3400]]

In [110]:
labels

[2187, 3188, 3189]

In [111]:
roi_dict = crop_object_to_dict(traced_image[-1], length=250, labels=labels)
with open(filename[:-4]+'.json', 'w') as json_file:
    json.dump(roi_dict, json_file, indent=4)

In [123]:
traced_image_stack = np.concatenate((image, traced_image[:,None,:,:]), axis=1)
imsave(filename[:-4]+"_track.tif", traced_image_stack, dtype=np.uint16, imagej=True)

In [124]:
dump_to_pkl(trace, filename[:-4]+".tracker")

In [213]:
# with open('data.json', 'r') as json_file:
#     loaded_data = json.load(json_file)
# print(loaded_data)